In [0]:
from pyspark.sql.functions import col, to_date, year, month

# Read bronze data with aliases
claims_df = spark.table("dev_catalog.default.bronze_hospital_dataset").alias("claims")
txn_df = spark.table("dev_catalog.default.bronze_transactions").alias("txn")

# Clean + type cast claims
typed_claims_df = (
    claims_df.dropna(subset=["ClaimID", "EncounterID"])
    .withColumn("Deductible", col("Deductible").cast("double"))
    .withColumn("Coinsurance", col("Coinsurance").cast("double"))
    .withColumn("Copay", col("Copay").cast("double"))
    .withColumn("PaidAmount", col("PaidAmount").cast("double"))
    .withColumn("ClaimStatus", col("ClaimStatus").cast("string"))
    .dropDuplicates(["ClaimID"])
    .alias("claims")
)

# Clean + type cast transactions
typed_txn_df = (
    txn_df.dropna(subset=["TransactionID", "EncounterID", "Amount"])
    .withColumn("Amount", col("Amount").cast("double"))
    .withColumn("TransactionDate", to_date(col("VisitDate"), "yyyy-MM-dd"))
    .dropDuplicates(["TransactionID"])
    .alias("txn")
)

# Join and select only needed columns with aliases
joined_df = (
    typed_claims_df.join(typed_txn_df, on="EncounterID", how="inner")
    .select(
        col("claims.ClaimID"),
        col("claims.EncounterID"),
        col("claims.Deductible"),
        col("claims.Coinsurance"),
        col("claims.Copay"),
        col("claims.PaidAmount"),
        col("claims.ClaimStatus"),
        col("txn.TransactionID"),
        col("txn.Amount"),
        col("txn.TransactionDate")
    )
    .withColumn("NetPayment", col("PaidAmount") - (col("Deductible") + col("Coinsurance") + col("Copay")))
    .withColumn("ClaimYear", year(col("TransactionDate")))
    .withColumn("ClaimMonth", month(col("TransactionDate")))
)

# Save as Delta table
joined_df.write.mode("overwrite").format("delta").saveAsTable("dev_catalog.default.Newsilver_claims_transactions")

